In [ ]:
from mca_tools import tools_map, query_user_for_details,gen_tools_desc
from mca_prompt import gen_prompt, user_prompt
tools_dict= gen_tools_desc()
print(tools_dict)

In [2]:
from mca_tools import tools_map, query_user_for_details
from mca_prompt import gen_prompt, user_prompt
from mca_model_provider import ModelProvider
from dotenv import load_dotenv
import dashscope
import os

load_dotenv()
mp = ModelProvider()

dashscope.api_key = "sk-f529539e3a50472fac783cf1e99fddef"
os.environ["TAVILY_API_KEY"] = "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"

def parse_thoughts(response, cur_request_time,max_request_time,debug):
    """解析模型返回的响应，提取关键信息并格式化为字符串，并根据debug参数决定是否打印"""
    try:
        thoughts = response.get("thoughts")
        planning = thoughts.get("planning")
        reasoning = thoughts.get("reasoning")
        reflection = thoughts.get("reflection")
        summary = thoughts.get("summary")
        observation = response.get("observation")
        prompt = f"planning: {planning}\nreasoning: {reasoning}\nreflection: {reflection}\nobservation: {observation}\nsummary: {summary}"
        prompt += f"\n这是医疗知识库第{cur_request_time}次响应，最多执行{max_request_time}次\n" 
        # if debug:
        #     print(prompt)
        return prompt
    except Exception as e:
        print(f"parse_thoughts error: {e}")
        return ""

def execute_action(action_name, action_args, debug):
    """根据动作名称执行相应的工具函数，并根据debug参数决定是否打印动作信息"""
    try:
        func = tools_map.get(action_name)
        result = func(**action_args)
        if debug:
            print(f"action_name: {action_name}, action_args: {action_args}")
        return result
    except Exception as e:
        print(f"调用工具异常： {e}")
        return str(e)

def agent_execute(query, max_request_time, debug=False):
    """执行代理任务，与模型交互并处理结果，根据debug参数决定是否打印详细信息"""
    cur_request_time = 0
    chat_history = []
    agent_scratch = ""

    while cur_request_time < max_request_time:
        cur_request_time += 1
        prompt = gen_prompt(query, agent_scratch)
        response = mp.chat(prompt, chat_history)

        if not response or not isinstance(response, dict):
            print(f"call llm exception, response is: {response}")
            continue

        action_info = response.get("action")
        action_name = action_info.get("name")
        action_args = action_info.get("args")
        thoughts = response.get("thoughts")

        if debug:
            print(f'-------------第{cur_request_time}次推断------------')
            print(f"observation: {response.get('observation')}")
            print(f"planning: {thoughts.get('planning')}")
            print(f"reasoning: {thoughts.get('reasoning')}")
            print(f"reflection: {thoughts.get('reflection')}")
            print(f"summary: {thoughts.get('summary')}")
            print(f"action_name: {action_name}, action_args: {action_args}")

        if action_name == "query_user_for_details":
            user_response = input(query_user_for_details(action_args["prompt"]))
            chat_history.append([action_args["prompt"], user_response])
            agent_scratch += f"query_user: {action_args['prompt']}user response: {user_response}"
            continue

        call_function_result = execute_action(action_name, action_args, debug)
        agent_scratch += f"observation: {response.get('observation')} execute action {action_name} result: {call_function_result}"
        assistant_msg = parse_thoughts(response, cur_request_time,max_request_time,debug)
        chat_history.append([user_prompt, assistant_msg])

        if action_name == "finish":
            final_answer = action_args.get("answer")
            # final_answer = "".join([str(key) + str(value) for key, value in final_answer.items()])
            break
    if cur_request_time == max_request_time:
        final_answer="本次任务执行失败！,未能提供问诊建议"
    return final_answer

# 示例执行，启用调试模式


In [ ]:
max_request_time = 20
query = '你好,我腱鞘炎又犯了，我应该具体吃什么药呢'
final_answer = agent_execute(query, max_request_time=max_request_time,debug=True)
print(f"final_answer: {final_answer}")

### final test